In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
%matplotlib inline
plt.style.use("seaborn-colorblind")
import sys, os

import pynoddy.history
import pynoddy.output
import pynoddy.experiment

import pymc

import theano.tensor as T
import GeoMig
import importlib
reload(GeoMig)
import importlib
import numpy as np
import pandas as pn
import pandas as pd
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import matplotlib
import matplotlib.cm as cm
from skimage import measure
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
np.set_printoptions(precision = 6, linewidth= 130, suppress =  True)
from mpl_toolkits.mplot3d import axes3d
import matplotlib.pyplot as plt
from matplotlib import cm
#%matplotlib inline
%matplotlib notebook
import pickle

### Initialization

Set up GeMpy model and load interface points, load and set gempy priors:

In [ ]:
nx = 400
nz = 200

GM = GeoMig.Interpolator(0,nx,
                         0,10,
                         0,nz,
                         u_grade=3)

# set grid resolution
GM.set_resolutions(nx,4,nz)
GM.create_regular_grid_3D()

# compile
GM.theano_compilation_3D()

# load gempy priors
prior_values = np.load("gempy_prior_values.npy")
# set priors
priors = [pymc.Normal(str(int(value[0]))+"_"+value[-1], value[2], 1./np.square(value[2]/5.)) for value in prior_values]
# load standard interfaces
interfaces = pickle.load( open( "gempy_interfaces.p", "rb" ) )

### PYMC gempy
##### Deterministic Functions

In [ ]:
@pymc.deterministic
def gempy_model(value=0, priors=priors, interf=interfaces, verbose=0, nx=nx, nz=nz):
    for i,value in enumerate(priors):
        #print value
        if value > 200:
            priors[i] = 199.
            #print priors[i]
    #print priors
    # set Z values in interface dataframe to prior values and load into gempy model object
    interf["Z"] = priors
    GM.load_data_pd("interfaces", interf)
    
    if verbose == 1:
        print GM.Interfaces
    
    # interpolate dip angles from wells and create foliation dataframe
    foliation_col_names = ["X", "Y", "Z", "azimuth", "dip", "polarity", "formation"]
    fol = pd.DataFrame(columns=foliation_col_names)
    for i in range(len(interf)-3):
        x = (interf.iloc[i]["X"] + interf.iloc[i+3]["X"])/2
        y = 0.
        z = (interf.iloc[i]["Z"] + interf.iloc[i+3]["Z"])/2
        dz = (interf.iloc[i]["Z"] - interf.iloc[i+3]["Z"])
        dx = (interf.iloc[i]["X"] - interf.iloc[i+3]["X"])
        tan = float(dz)/float(dx)
        dip = np.rad2deg(np.arctan(tan))
        az = 90.
        pol = 1.
        fol.loc[i] = [x,y,z,az,-1*dip,pol,interf.iloc[i]["formation"]]
    GM.load_data_pd("foliations", fol)
    
    if verbose == 1:
        print GM.Foliations
    
    # load layers into series
    initial = []
    for layer in GM.formations:
        initial.append(layer)
    GM.set_series({"Initial": tuple(initial)})
    
    if verbose==1:
        print GM.series
    
    # gempy magic calculation
    GM.block.set_value(np.zeros_like(GM.grid[:,0]))
    GM.compute_block_model([0], verbose = 0)
    plot_block =  GM.block.get_value().reshape(nx,4,nz)
    
    # return gempy block model
    if verbose == 1:
        plt.imshow(plot_block[:,2,:].T, origin="lower")
        plt.plot(GM.Interfaces["X"],GM.Interfaces["Z"],"o")
        plt.plot(GM.Foliations["X"],GM.Foliations["Z"],"x")
        plt.xlim(0,400)
        plt.ylim(0,200)
    return plot_block[:,2,:]

#### Likelihood Functions

In [ ]:
# load from pickled file
like_layer_heights = pickle.load( open( "like_layer_heights.p", "rb" ) )

# layer 1
@pymc.stochastic
def like_xpos0_layer_1(value=0, gempy_model=gempy_model, like_layer_heights=like_layer_heights):
    x_pos=like_layer_heights.keys()
    if 1.0 in gempy_model[x_pos[0],:]:
        model_height = np.where(gempy_model[x_pos[0],:]==1.0)[0][0]
        return np.log(like_layer_heights[x_pos[0]][1].evaluate(model_height))[0]
    else:
        return np.log(0.001)

@pymc.stochastic
def like_xpos1_layer_1(value=0, gempy_model=gempy_model, like_layer_heights=like_layer_heights):
    x_pos=like_layer_heights.keys()
    if 1.0 in gempy_model[x_pos[1],:]:
        model_height = np.where(gempy_model[x_pos[1],:]==1.0)[0][0]
        return np.log(like_layer_heights[x_pos[1]][1].evaluate(model_height))[0]
    else:
        return np.log(0.001)
    
@pymc.stochastic
def like_xpos2_layer_1(value=0, gempy_model=gempy_model, like_layer_heights=like_layer_heights):
    x_pos=like_layer_heights.keys()
    if 1.0 in gempy_model[x_pos[2],:]:
        model_height = np.where(gempy_model[x_pos[2],:]==1.0)[0][0]
        return np.log(like_layer_heights[x_pos[2]][1].evaluate(model_height))[0]
    else:
        return np.log(0.001)
    
@pymc.stochastic
def like_xpos3_layer_1(value=0, gempy_model=gempy_model, like_layer_heights=like_layer_heights):
    x_pos=like_layer_heights.keys()
    if 1.0 in gempy_model[x_pos[3],:]:
        model_height = np.where(gempy_model[x_pos[3],:]==1.0)[0][0]
        return np.log(like_layer_heights[x_pos[3]][1].evaluate(model_height))[0]
    else:
        return np.log(0.001)

@pymc.stochastic
def like_xpos4_layer_1(value=0, gempy_model=gempy_model, like_layer_heights=like_layer_heights):
    x_pos=like_layer_heights.keys()
    if 1.0 in gempy_model[x_pos[4],:]:
        model_height = np.where(gempy_model[x_pos[4],:]==1.0)[0][0]
        return np.log(like_layer_heights[x_pos[4]][1].evaluate(model_height))[0]
    else:
        return np.log(0.001)
    
# -----------------------------------------------------------------------------------
# -----------------------------------------------------------------------------------
# -----------------------------------------------------------------------------------
# layer 2
@pymc.stochastic
def like_xpos0_layer_2(value=0, gempy_model=gempy_model, like_layer_heights=like_layer_heights):
    x_pos=like_layer_heights.keys()
    if 2.0 in gempy_model[x_pos[0],:]:
        model_height = np.where(gempy_model[x_pos[0],:]==2.0)[0][0]
        return np.log(like_layer_heights[x_pos[0]][2].evaluate(model_height))[0]
    else:
        return np.log(0.001)

@pymc.stochastic
def like_xpos1_layer_2(value=0, gempy_model=gempy_model, like_layer_heights=like_layer_heights):
    x_pos=like_layer_heights.keys()
    if 2.0 in gempy_model[x_pos[1],:]:
        model_height = np.where(gempy_model[x_pos[1],:]==2.0)[0][0]
        return np.log(like_layer_heights[x_pos[1]][2].evaluate(model_height))[0]
    else:
        return np.log(0.001)
    
@pymc.stochastic
def like_xpos2_layer_2(value=0, gempy_model=gempy_model, like_layer_heights=like_layer_heights):
    x_pos=like_layer_heights.keys()
    if 2.0 in gempy_model[x_pos[2],:]:
        model_height = np.where(gempy_model[x_pos[2],:]==2.0)[0][0]
        return np.log(like_layer_heights[x_pos[2]][2].evaluate(model_height))[0]
    else:
        return np.log(0.001)
    
@pymc.stochastic
def like_xpos3_layer_2(value=0, gempy_model=gempy_model, like_layer_heights=like_layer_heights):
    x_pos=like_layer_heights.keys()
    if 2.0 in gempy_model[x_pos[3],:]:
        model_height = np.where(gempy_model[x_pos[3],:]==2.0)[0][0]
        return np.log(like_layer_heights[x_pos[3]][2].evaluate(model_height))[0]
    else:
        return np.log(0.001)
    
@pymc.stochastic
def like_xpos4_layer_2(value=0, gempy_model=gempy_model, like_layer_heights=like_layer_heights):
    x_pos=like_layer_heights.keys()
    if 2.0 in gempy_model[x_pos[4],:]:
        model_height = np.where(gempy_model[x_pos[4],:]==2.0)[0][0]
        return np.log(like_layer_heights[x_pos[4]][2].evaluate(model_height))[0]
    else:
        return np.log(0.001)
    
# -----------------------------------------------------------------------------------
# -----------------------------------------------------------------------------------
# -----------------------------------------------------------------------------------
# layer 3
@pymc.stochastic
def like_xpos0_layer_3(value=0, gempy_model=gempy_model, like_layer_heights=like_layer_heights):
    x_pos=like_layer_heights.keys()
    if 3.0 in gempy_model[x_pos[0],:]:
        model_height = np.where(gempy_model[x_pos[0],:]==3.0)[0][0]
        return np.log(like_layer_heights[x_pos[0]][3].evaluate(model_height))[0]
    else:
        return np.log(0.001)

@pymc.stochastic
def like_xpos1_layer_3(value=0, gempy_model=gempy_model, like_layer_heights=like_layer_heights):
    x_pos=like_layer_heights.keys()
    if 3.0 in gempy_model[x_pos[1],:]:
        model_height = np.where(gempy_model[x_pos[1],:]==3.0)[0][0]
        return np.log(like_layer_heights[x_pos[1]][3].evaluate(model_height))[0]
    else:
        return np.log(0.001)
    
@pymc.stochastic
def like_xpos2_layer_3(value=0, gempy_model=gempy_model, like_layer_heights=like_layer_heights):
    x_pos=like_layer_heights.keys()
    if 3.0 in gempy_model[x_pos[2],:]:
        model_height = np.where(gempy_model[x_pos[2],:]==3.0)[0][0]
        return np.log(like_layer_heights[x_pos[2]][3].evaluate(model_height))[0]
    else:
        return np.log(0.001)

@pymc.stochastic
def like_xpos3_layer_3(value=0, gempy_model=gempy_model, like_layer_heights=like_layer_heights):
    x_pos=like_layer_heights.keys()
    if 3.0 in gempy_model[x_pos[3],:]:
        model_height = np.where(gempy_model[x_pos[3],:]==3.0)[0][0]
        return np.log(like_layer_heights[x_pos[3]][3].evaluate(model_height))[0]
    else:
        return np.log(0.001)
    
@pymc.stochastic
def like_xpos4_layer_3(value=0, gempy_model=gempy_model, like_layer_heights=like_layer_heights):
    x_pos=like_layer_heights.keys()
    if 3.0 in gempy_model[x_pos[4],:]:
        model_height = np.where(gempy_model[x_pos[4],:]==3.0)[0][0]
        return np.log(like_layer_heights[x_pos[4]][3].evaluate(model_height))[0]
    else:
        return np.log(0.001)


In [ ]:
like_list = [like_xpos0_layer_1,like_xpos1_layer_1,like_xpos2_layer_1,like_xpos3_layer_1,like_xpos4_layer_1,
             like_xpos0_layer_2,like_xpos1_layer_2,like_xpos2_layer_2,like_xpos3_layer_2,like_xpos4_layer_2,
             like_xpos0_layer_3,like_xpos1_layer_3,like_xpos2_layer_3,like_xpos3_layer_3,like_xpos4_layer_3]

In [ ]:
params = [priors[i] for i in range(len(priors))]
params.append(gempy_model)
for entry in like_list:
    params.append(entry)
model = pymc.Model(params)

In [ ]:
iterations =

In [ ]:
RUN = pymc.MCMC(model)
RUN.sample(iter=iterations)